# wps_CI_on_demand_with_indices

#### wps_CI is a process that runs the [ci.netcdf.wrapper](https://github.com/pacificclimate/ClimDown/blob/master/R/CI.R#L235) (Climate Imprint) function of the [ClimDown](https://cran.r-project.org/web/packages/ClimDown/index.html) package using the [`chickadee`](https://github.com/pacificclimate/chickadee) DACCS bird. This notebook uses either the PNWNAmet daily gridded observations or the CMIP6 downscaled data as the `gcm_file` parameter and the PRISM monthly climatologies as the `obs_file` parameter. By supplying climatologies, this skips the computation of the climatologies during Climate Imprint. After running CI, the output can be passed to the Ouranos [`finch`](https://github.com/bird-house/finch/tree/master) bird to compute climate indices. This notebook uses an interactive map to set up inputs to the birds and run them, and its setup process is detailed in the `map.py` module.

#### You can also examine pre-computed `chickadee` and `finch` outputs in steps VI and IX. The `chickadee` outputs can be used as inputs to the `finch` processes so that you don't have to use the interactive map if you only wish to compute climate indices.

I. Import `map` module and its functions/variables.

In [1]:
import map
from map import *

II. Check `url`s used to connect to chickadee/finch.

In [2]:
# NBVAL_IGNORE_OUTPUT
print(f"Using chickadee on {chickadee_url}")
print(f"Using finch on {finch_url}")

Using chickadee on http://docker-dev03.pcic.uvic.ca:30102
Using finch on https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/finch/wps


III. Display help for individual processes by using the ? command (ex. bird.process?). We can use the docstring to ensure we provide the appropriate parameters.

In [3]:
# NBVAL_IGNORE_OUTPUT
chickadee.ci?

Signature:
chickadee.ci(
    gcm_file,
    obs_file,
    gcm_varname=None,
    obs_varname=None,
    num_cores='4',
    loglevel='INFO',
    units_bool=True,
    n_pr_bool=True,
    tasmax_units='celsius',
    tasmin_units='celsius',
    pr_units='kg m-2 d-1',
    max_gb=1.0,
    start_date=datetime.date(1971, 1, 1),
    end_date=datetime.date(2005, 12, 31),
    out_file=None,
)
Docstring:
Climate Imprint (CI) downscaling

Parameters
----------
gcm_file : ComplexData:mimetype:`application/x-netcdf`, :mimetype:`application/x-ogc-dods`
    Filename of GCM simulations
obs_file : ComplexData:mimetype:`application/x-netcdf`, :mimetype:`application/x-ogc-dods`
    Filename of high-res gridded historical observations
out_file : string
    Filename to create with the climate imprint outputs
num_cores : {'1', '2', '3', '4'}positiveInteger
    The number of cores to use for parallel execution
loglevel : {'CRITICAL', 'ERROR', 'WARNING', 'INFO', 'DEBUG', 'NOTSET'}string
    Logging level
units_boo

In [4]:
# NBVAL_IGNORE_OUTPUT
finch.tx_max?

Signature:
finch.tx_max(
    tasmax=None,
    freq='YS',
    month=None,
    season=None,
    check_missing='any',
    missing_options=None,
    cf_compliance='warn',
    data_validation='raise',
    variable=None,
    output_name=None,
    output_format='netcdf',
    csv_precision=None,
    output_formats=None,
)
Docstring:
The maximum value of daily maximum temperature.

Parameters
----------
tasmax : ComplexData:mimetype:`application/x-netcdf`, :mimetype:`application/x-ogc-dods`
    NetCDF Files or archive (tar/zip) containing netCDF files. Maximum surface temperature.
freq : {'YS', 'MS', 'QS-DEC', 'AS-JUL'}string
    Resampling frequency.
month : {'1', '2', '3', '4', '5', '6', '7', '8', '9', '10', ...}integer
    Months of the year over which to compute indicator.
season : {'DJF', 'MAM', 'JJA', 'SON'}string
    Climatological season over which to compute indicator.
check_missing : {'any', 'wmo', 'pct', 'at_least_n', 'skip', 'from_context'}string
    Method used to determine which a

IV. Load the interactive map to initialize the inputs. These inputs are the <b>center point</b> of the 2.5 degree by 2.5 degree subdomain of the target region and 3 degree by 3 degree subdomain of the GCM region, the <b>climate variable</b> from the GCM input to downscale, the <b>dataset</b> to use as the GCM input (where choosing the CMIP6 data allows one to further select the <b>downscaling technique</b>, the <b>model</b>, the <b>CanESM5 run</b> if applicable, and the <b>emissions scenario</b>), and the <b>climatological period</b> of the target data. Some inputs are obtained from the [THREDDS data server](https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/catalog.html). Once the inputs are initialized, you can click the `Run Chickadee` button to start the CI process.

Alternatively, if you wish to inspect pre-computed outputs from `chickadee`, you can skip to <b>step VI</b> and run the `use_default_chickadee_outputs` cell. 

In [5]:
AppLayout(center = m, right_sidebar = control_box_chickadee, align_items = 'center')

AppLayout(children=(Box(children=(Text(value='', placeholder=''), Text(value='', description='Center:', placeh…

V. Run this cell to display `chickadee` progress.

In [6]:
ipydisplay.display(output_widget_chickadee)

Output()

VI. Check `chickadee` statuses and store the outputs as `netCDF4.Dataset` objects if possible. `map.chickadee_outputs` is used to ensure that the updated values rather than the default `None` values are passed to these functions.

Alternatively, one can run the `use_default_chickadee_outputs` cell to obtain pre-computed `chickadee` outputs.

In [7]:
use_default_chickadee_outputs()

In [8]:
for var in map.chickadee_outputs.keys():
    for chickadee_output in map.chickadee_outputs[var]:
        get_bird_output(chickadee_output)
        if chickadee_output.isComplete():
            chickadee_output_ds = bird_output_to_dataset(chickadee_output)
            print(chickadee_output_ds)

Process status: ProcessSucceeded
Link to process output: https://docker-dev03.pcic.uvic.ca/wpsoutputs/ef45dbfc-dcac-11ee-a97c-0242ac12000c/pr_PNWNAmet_target_198101-201012_on-demand.nc
<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format DAP2):
    DODS_EXTRA.Unlimited_Dimension: time
    dimensions(sizes): time(24837), lat(301), lon(301)
    variables(dimensions): float64 lon(lon), float64 lat(lat), float64 time(time), float32 pr(time, lat, lon)
    groups: 
Process status: ProcessSucceeded
Link to process output: https://docker-dev03.pcic.uvic.ca/wpsoutputs/f099db66-dcac-11ee-a97c-0242ac12000c/tasmax_PNWNAmet_target_198101-201012_on-demand.nc
<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format DAP2):
    DODS_EXTRA.Unlimited_Dimension: time
    dimensions(sizes): time(24837), lat(301), lon(301)
    variables(dimensions): float64 lon(lon), float64 lat(lat), float64 time(time), float32 tasmax(time, lat, lon)
    group

VII. Once `chickadee` processes are complete, select output files from which `finch` will compute climate indices based on the climate variable in each file. Currently, a subset of the [climdex indices](https://climate-scenarios.canada.ca/?page=climdex-indices) can be computed with more to be available in the future.

Similarly to `chickadee`, you can instead examine pre-computed `finch` outputs by skipping to <b>step IX</b> and running the `use_default_finch_outputs` cell.

In [9]:
display_chickadee_outputs()
display(map.chickadee_output_box, finch_indices, run_finch)

Button(button_style='info', description='Run Finch', style=ButtonStyle(), tooltip="Click 'Run' to start the cl…

VIII. Run this cell to display `finch` progress.

In [10]:
ipydisplay.display(output_widget_finch)

Output()

IX. As for `chickadee`, check the statuses of the `finch` processes and store the outputs as `netCDF4.Dataset` objects if possible.

Alternatively, one can run the `use_default_finch_outputs` cell to obtain pre-computed finch outputs.

In [12]:
use_default_finch_outputs()

In [13]:
finch_outputs_ds = []
for output in map.finch_outputs:
    get_bird_output(output)
    if output.isComplete():
        finch_output_ds = bird_output_to_dataset(output)
        print(finch_output_ds)
        finch_outputs_ds.append(finch_output_ds)

Process status: ProcessSucceeded
Link to process output: https://docker-dev03.pcic.uvic.ca/wpsoutputs/finch/69935d0a-ead9-11ee-8fa3-0242ac120004/rx2day_monthly.nc
<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format DAP2):
    DODS_EXTRA.Unlimited_Dimension: time
    climateindex_package_id: https://github.com/Ouranosinc/xclim
    product: derived climate index
    institute_id: CCCS
    contact: Canadian Centre for Climate Services
    institution: Canadian Centre for Climate Services (CCCS)
    dimensions(sizes): time(816), lat(301), lon(301)
    variables(dimensions): float32 time(time), float64 lat(lat), float64 lon(lon), float32 rx2day(time, lat, lon)
    groups: 
Process status: ProcessSucceeded
Link to process output: https://docker-dev03.pcic.uvic.ca/wpsoutputs/finch/6a58b85c-ead9-11ee-8fa3-0242ac120004/r18mm_djf.nc
<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format DAP2):
    DODS_EXTRA.Unlimited_Dimension: 